In [1]:
#Run this cell

from datascience import *
from pandas import read_stata
import numpy as np
from pygrowup import Calculator

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

#Use the data that you imported at the beginning of Lab 4  
data = Table.read_table("***Appropriate file path to Lab 3's CSV***")
data

OSError: File b"***Appropriate file path to Lab 3's CSV***" does not exist

In [ ]:
# If you need to join in extra data from earlier because you lost a lot of observations from too few birthdates, do so here.
# Copy the code you developed in Lab 4.

# Household Composition II

This week we will look at specific information about parents. Examine your data and find a household with a child that has both a mother and a father living in the household.  This means both Mother ID and Father ID are not empty.  Just looking at this one family, can you determine mother's age and father's age? How about mother's education and father's education? Hint: Be sure to use the correct education variable (likely 'Years of Schooling' rather than 'Currently Enrolled in School'.)  

Challenge yourself and try this activity with the kids who are living in the largest household in your data.

### Additional Preparation

You probably have two or more schooling variables.  Combine them into a single column "Education" to simplify your data.  

Note: some data sets will have two variables for level of schooling and year. For example, level is high school & years is 3. This person's years of schooling should be 11. The code below illustrates what transformation you might have to do. Check your questionnaire!

In [ ]:
#check your questionnaire for coding values
def yearsextra(x):
    #preschool
    if x==1: 
        return 0
    #lower primary
    elif x==2: 
        return 0
    #upper primary
    elif x==3: 
        return 6
    #lower secondary
    #etc. 
    #etc.
    else:
        return -99
  
data['Extra Years']=data.apply(yearsextra, 'Years of Schooling - Level')
data['Education Level']=data['Years of Schooling - Years']+data['Extra Years']

# You may need to do this a second time for Currently Enrolled in School

#Make a new column for Education that includes years of schooling for those enrolled & those who have completed.
data['Education Level']= [data['***Appropriate Column Name***'][i] if data['***Appropriate Column Name***'][i] > 0 else data['***Appropriate Column Name***'][i] for i in range(data.num_rows)]


We can do a quick check that the schooling makes sense and that people do not have more years of education than years of life.  Do a scatter plot comparing Age in years to Education (without the -99 values).

Now that your Education data looks good, drop the other education columns so you don't get confused ('Extra Years', 'Years of Schooling - Years', 'Years of Schooling - Level').  Keep 'Currently Enrolled'.

We will now join in information about mothers.

In [ ]:
# Make a Mother Master ID: 
data['Mother Master ID']=data['Household ID']*1000+data['Mother ID']

# Make a table of information we want about mothers
data_on_moms=data.select('Household ID', 'Individual ID', 'Age in Years', 'Education')
data_on_moms=data_on_moms.relabel(['Age in Years', 'Education'],["Mom's Age", "Mom's Education"])

# Make a Mother Master ID: 
data_on_moms['Mother Master ID']=data['Household ID']*1000+data['Individual ID']

# Create a table with rows all -99 for children who do not have mothers in the household
data_on_moms_small=data_on_moms.group("Household ID")
data_on_moms_small=data_on_moms_small.select("Household ID")
data_on_moms_small['Individual ID']=-99
data_on_moms_small["Mom's Age"]=-99
data_on_moms_small["Mom's Education"]=-99
data_on_moms_small['Mother Master ID']=data_on_moms_small['Household ID']*1000+data_on_moms_small['Individual ID']

#append households with no mother IDs to created mother IDs
data_on_moms=data_on_moms.append(data_on_moms_small)
data_on_moms=data_on_moms.drop('Household ID', 'Individual ID')
data_on_moms

# Join the mother information to the child using the Mother Master ID
data=data.join('Mother Master ID', data_on_moms, 'Mother Master ID')
data=data.drop('Mother Master ID')
data.show(100)

Sometimes joining brings in extra lines of data.  Make sure you still have the same number of people that you started out with in the original table.

Examine your data.  Do you now have information about mothers for children who have mothers?  Does it make sense? (i.e. generally were children born when their mother was older than age 15.)  Make sure you do not have information about mothers for individuals that do not have mothers in the household.  

<font color="Blue"> Item 0: Make a new variable: Mother's age when child was born.  What is the youngest age at which a mother had a baby in your data set?

Do the same for fathers: Dad's Age, Dad's Education, and Dad's Relation to Head

In Lab 4 we looked at nuclear families by checking if members were children of the head of the household. This method leaves out the possibility of step-children.  To be sure we get nuclear families and not blended families, we need to make sure the children are children of both the head and the spouse of the head.  <font color="Blue"> Item 1: What percentage of children are children with married parents (and one is the household head.)  Create a new variable "Nuclear (of Head)"

<font color="Blue"> Item 2:  This does not exactly fit with the nuclear family definition.  What else would you need to do ensure only nuclear families and all nuclear families. (Don't code, just describe.) 

In [ ]:
Save your data.

## Let's examine if parent's education is associated with child outcomes.

Today's article suggests mother's eduction is better correlated with nutrition than father education.  <font color="Blue"> Item 3:  Make a scatter plot of z-scores by father's education level.  Make another by mother's education level. Do your initial findings align with the article? (Be sure to only graph the children and eliminate the missing values.)

<font color="Blue"> Item 4: Let's see if we get a clearer picture by using mean: Calculate mean z-scores by mothers' and fathers' education levels.  Graph these and comment on your findings. 

Z-scores are just for kids age 5 and under.  Let's check out how the older kids are doing. 
<font color="Blue"> Item 5:  Calculate percent of 14-18 year olds currently enrolled in school by mother's and father's education level.

<font color="Black"> Hint: After confirming enrollment values, make a small table only including children of those ages and then pivot enrollment around mother's and father's education levels.

## Big challenge!
Can you determine the age and education of grandparents?  Specifically, maternal grandmother, maternal grandfather, paternal grandmother, and paternal grandfather.  Hint: what if earlier you also included Mother's Mother's ID in the mother data base used for joining.  Again, try looking at the largest family to see how this works before attempting the coding.